# Notebook 1) Data Acquisition & SQL Loading

This notebook implements the **data ingestion and storage layer** of the project.  
Its goal is to create a reproducible and structured foundation for all the following analysis and modeling.

Project workflow: 

1. **Ingestion** → bring raw data into a reliable storage database  
2. **Transformation** → clean, filter, and engineer features
3. **Modeling** → statistical inference + ML prediction  
4. **Deployment** → deploy model through a Streamlit application  

Notebook 1 covers **Step 1** only.

## What this notebook does  
- Loads the CPS CSV into memory  
- Performs a small renaming step to avoid SQL column-name conflicts  
- Creates (or connects to) a SQLite database  
- Stores the dataset as a **raw table `cps_raw`**  
- Verifies the ingestion with preview queries  

No filtering, cleaning, or modeling occurs here as those steps are intentionally executed in *Notebook 2* and in the SQL script `01_load_curate.sql` where the modeling-ready view `vw_model_cohort` is generated.

# 1) Importing libraries and setting paths

In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
db_path = "../data/sql_cps_database.db"
csv_path = "../data/CurrentPopulationSurvey.csv"

print("Working directory: ", os.getcwd())
print("CSV path: ", csv_path)
print("Database path: ", db_path)

Working directory:  /Users/sfia/Desktop/Business_Final_Project/notebooks
CSV path:  ../data/CurrentPopulationSurvey.csv
Database path:  ../data/sql_cps_database.db


# 2) Sanity check on files

In [3]:
for x in [db_path, csv_path]:
    status = "OK" if os.path.exists(x) else "MISSING"
    print(f"{x} -> {status}")

../data/sql_cps_database.db -> OK
../data/CurrentPopulationSurvey.csv -> OK


## 3) Load CSV + Quick Overview

In [4]:
df = pd.read_csv(csv_path)
print(f"Number of Rows: {df.shape[0]} \nNumber of Columns: {df.shape[1]}")
df.head(10)

Number of Rows: 344287 
Number of Columns: 234


,year,serial,numprec,hwtsupp,gq,region,statefip,metro,metarea,county,...,foodcare,building,sales,officeadmin,farmer,constructextractinstall,production,transport,sumadj_occ,LEHS
0,1990,9677,1,1052.650024,1,12,36,2.0,5607.0,NaN,...,0,0,0,1,0,0,0,0,1,1
1,2009,64923,4,971.200012,1,33,5,2.0,NaN,NaN,...,0,0,0,1,0,0,0,0,1,1
2,1990,8425,1,1622.280029,1,12,36,3.0,5607.0,NaN,...,0,0,0,0,0,0,0,0,1,1
3,1990,67963,1,2689.909912,1,42,6,3.0,7400.0,NaN,...,0,0,0,0,0,0,0,0,1,0
4,1981,38650,9,1911.900024,1,31,51,3.0,8840.0,NaN,...,0,0,0,0,0,0,0,0,1,1
5,1999,63841,5,1858.479980,1,42,6,2.0,680.0,6029.0,...,0,0,0,0,0,0,0,0,1,1
6,2007,25381,5,1488.300049,1,21,17,3.0,1605.0,NaN,...,0,0,0,0,0,0,1,0,1,0
7,1990,56727,2,1889.739990,1,33,48,3.0,1920.0,NaN,...,0,0,1,0,0,0,0,0,1,1
8,1999,28603,4,609.080017,1,31,10,4.0,9160.0,10003.0,...,0,0,0,1,0,0,0,0,1,1
9,2011,5146,2,170.889999,1,11,50,4.0,1311.0,NaN,...,0,0,0,0,0,0,0,1,1,1


In [5]:
df = df.rename(columns={"Transport": "transport_ind", "transport": "transport_occ"})

print("Renamed columns to avoid SQLite name collision:")
print([col for col in df.columns if "transport" in col.lower()])

Renamed columns to avoid SQLite name collision:
['transport_ind', 'transport_occ']


## 4) Create/connect to SQLite Database

In [6]:
conn = sqlite3.connect(db_path)

## 5) Write to SQLite as cps_raw

In [7]:
table_name= "cps_raw"

df.to_sql(table_name, conn, if_exists="replace", index=False)

print(f"Loaded RAW CPS into sqlite as '{table_name}' in {db_path}")

Loaded RAW CPS into sqlite as 'cps_raw' in ../data/sql_cps_database.db


## 6) Quick sanity query from SQLite

In [8]:
preview_sql = pd.read_sql("SELECT * FROM cps_raw LIMIT 5", conn)
count_sql = pd.read_sql("SELECT COUNT(*) AS n_rows FROM cps_raw", conn)

print("Preview of cps_raw:")
display(preview_sql)

print("Row count:")
display(count_sql)

Preview of cps_raw:


,year,serial,numprec,hwtsupp,gq,region,statefip,metro,metarea,county,...,foodcare,building,sales,officeadmin,farmer,constructextractinstall,production,transport_occ,sumadj_occ,LEHS
0,1990,9677,1,1052.650024,1,12,36,2.0,5607.0,None,...,0,0,0,1,0,0,0,0,1,1
1,2009,64923,4,971.200012,1,33,5,2.0,NaN,None,...,0,0,0,1,0,0,0,0,1,1
2,1990,8425,1,1622.280029,1,12,36,3.0,5607.0,None,...,0,0,0,0,0,0,0,0,1,1
3,1990,67963,1,2689.909912,1,42,6,3.0,7400.0,None,...,0,0,0,0,0,0,0,0,1,0
4,1981,38650,9,1911.900024,1,31,51,3.0,8840.0,None,...,0,0,0,0,0,0,0,0,1,1


Row count:


,n_rows
0,344287


In [9]:
conn.close()

### End of Notebook 1